# Satellite tracking Animation

We create an animation of the satellite and the lightnings it encounters on its path.

In [17]:
# Import libraries
import raadpy as rp
from tqdm.notebook import tqdm

# import the location file
filename = '../../Data/Location/LIGHT-1_5PP.csv'

# Load this data inot a raadpy array
data = rp.array(filename=filename,event_type='location')

# Get a subset of this data
path = rp.array(data[:1000])

# Find the lightnings around this path
timestamps  = path.get_timestamps()
t_min_idx   = timestamps.argmin()
t_min       = timestamps[t_min_idx]
t_max_idx   = timestamps.argmax()
t_max       = timestamps[t_max_idx]

# Collect the lightnings
lights = rp.download_lightnings_range(t_min,t_max,VERBOSE=True)
len(lights)

Found Lightning data at: https://www.blitzortung.org/en/archive_data.php?stations_users=0&selected_numbers=*&end_date=1652227200&end_time=51597&start_date=1652227200&start_time=31617&rawdata_image=0&north=90&west=-180&east=180&south=-90&map=0&width_orig=640&width_result=640&agespan=60&frames=12&delay=100&last_delay=1000&show_result=1
Data Downloaded Successfully


72951

In [18]:
# Animate based on timestamps
import plotly.graph_objects as go
size = 500
long = -90
lat  = 30

# Create the map
fig = go.Figure(data = go.Scattergeo(
    name='Cubesat Location',
    lon=path.get_coords().T[0],
    lat=path.get_coords().T[1],
    text=path.get_timestamps(format='iso'),
    # animation_frame=path.get_timestamps(format='mjd'),
    mode = 'markers',
    marker=dict(
        size=4,
        color='blue',
        symbol = 'circle-dot'
    ),
))

fig.update_geos(projection_type="orthographic",projection_rotation=dict(lon=long, lat=lat))
fig.update_layout(height=size,width=size, margin={"r":0,"t":0,"l":0,"b":0})

In [28]:
# First we will create the frames
frames = []
path_times  = path.get_timestamps(format='unix')
path_iso    = path.get_timestamps(format='iso')
path_xy     = path.get_coords().T
light_times = lights.get_timestamps(format='unix')
light_iso   = lights.get_timestamps(format='iso')
light_xy    = lights.get_coords().T
THRESHOLD = 1

# Define an opacity function
opacity = lambda x: np.exp(-(x/THRESHOLD)**2)

for i in tqdm(range(len(path_times))):

    # Find the closest lightnigs of this timestamp
    close_idx = np.where(abs(path_times[i] - light_times) <= THRESHOLD)[0]

    frames.append(dict(data=[go.Scattergeo(name='Cubesat Location',
        lon=[path_xy[0][i]],
        lat=[path_xy[1][i]],
        text=path_iso[i],
        mode = 'markers',
        marker=dict(
            size=8,
            color='blue',
            symbol = 'circle-dot'
        ))]))

    if len(close_idx) > 0:
        frames[-1]['data'].append(
            go.Scattergeo(name='lightnings',
                lon=light_xy[0][close_idx],
                lat=light_xy[1][close_idx],
                text=light_iso[close_idx],
                opacity = opacity(path_times[i] - light_times[close_idx]),
                mode = 'markers',
                marker=dict(
                    size=8,
                    color='red',
                    symbol = 'circle-dot'
        )))

# Play button
play_button = [{
    'type':'buttons',
    'showactive':True,
    'x':0.045, 'y':-0.08,
    'buttons':[{ 
        'label':'🎬', # Play
        'method':'animate',
        'args':[
            None,
            {
                'frame':{'duration':100, 'redraw':True},
                'transition':{'duration':50},
                'fromcurrent':True,
                'mode':'immediate',
            }
        ]
    }]
}]

data = frames[0]['data']

# Adding all sliders and play button to the layout
layout = go.Layout(
    # sliders=sliders,
    updatemenus=play_button
)

# Creating the figure
fig = go.Figure(data=data, layout=layout, frames=frames)
fig.update_geos(projection_type="orthographic",projection_rotation=dict(lon=long, lat=lat))
fig.update_layout(height=size,width=size, margin={"r":0,"t":0,"l":0,"b":0})

# Displaying the figure
fig.show()

  0%|          | 0/1000 [00:00<?, ?it/s]

In [27]:
import numpy as np

np.where(abs(lights.get_timestamps(format='unix') - path[7].get_timestamp('unix')) < 0.5)

(array([], dtype=int64),)